# 🎧 EDA Spotify Tracks 2023 (R)

Autor: José Manuel Sánchez Álvarez  
Centro: IES Rafael Alberti — Curso 2025–2026  

Exploración de datos del dataset *Spotify Tracks 2023* usando **R y tidyverse**.

Este notebook realiza un análisis exploratorio (EDA) para estudiar popularidad, duración, energía, correlaciones y géneros musicales.


## 1️⃣ Preparación del entorno

Se cargan las librerías necesarias y se configuran los gráficos inline.

In [ ]:
suppressPackageStartupMessages({
  if (!requireNamespace("tidyverse", quietly = TRUE)) install.packages("tidyverse", repos = "https://cloud.r-project.org")
  if (!requireNamespace("ggcorrplot", quietly = TRUE)) install.packages("ggcorrplot", repos = "https://cloud.r-project.org")
  library(tidyverse)
  library(ggcorrplot)
})
options(repr.plot.width = 7, repr.plot.height = 4)
sessionInfo()


: 

## 2️⃣ Carga del dataset

Se intentará descargar automáticamente desde una fuente pública. Si falla, se utilizará un archivo local `spotify_tracks_2023.csv`.

In [ ]:
url <- "https://raw.githubusercontent.com/marceloreisjr/datasets/master/spotify_tracks_2023.csv"
spotify <- tryCatch({
  read_csv(url)
}, error = function(e) {
  message("Descarga fallida. Cargando archivo local...")
  read_csv("spotify_tracks_2023.csv")
})

glimpse(spotify)
head(spotify)


: 

## 3️⃣ Limpieza básica

- Eliminamos columnas irrelevantes (`Unnamed:_0` si existe).
- Quitamos duplicados.
- Comprobamos valores nulos.

In [ ]:
if ("Unnamed:_0" %in% colnames(spotify)) {
  spotify <- spotify %>% select(-`Unnamed:_0`)
}

spotify <- spotify %>% distinct()

colSums(is.na(spotify)) %>% sort(decreasing = TRUE)


## 4️⃣ Estadísticas descriptivas

- Popularidad
- Duración (convertida a minutos)
- Valores de energía, valence, danceability, etc.

In [ ]:
spotify <- spotify %>% mutate(duration_min = duration_ms / 60000)

summary(select(spotify, popularity, duration_min, danceability, energy, valence, tempo))


## 5️⃣ Visualizaciones básicas

### 5.1 Distribución de popularidad

In [ ]:
ggplot(spotify, aes(x = popularity)) +
  geom_histogram(binwidth = 5, fill = "steelblue", color = "white") +
  labs(title = "Distribución de popularidad", x = "Popularidad", y = "Frecuencia")


### 5.2 Duración de canciones (en minutos)

In [ ]:
ggplot(spotify, aes(y = duration_min)) +
  geom_boxplot(fill = "lightgreen") +
  labs(title = "Duración de canciones (minutos)", y = "Minutos")


## 6️⃣ Correlaciones entre variables numéricas

In [ ]:
num_cols <- spotify %>% select(popularity, danceability, energy, loudness, speechiness,
                               acousticness, instrumentalness, liveness, valence, tempo)

corr_matrix <- cor(num_cols, use = "complete.obs")
ggcorrplot(corr_matrix, hc.order = TRUE, type = "lower", lab = TRUE, lab_size = 3,
           colors = c("red", "white", "blue"), title = "Mapa de correlaciones entre variables")


## 7️⃣ Popularidad media por género

In [ ]:
genre_stats <- spotify %>%
  group_by(track_genre) %>%
  summarise(media_pop = mean(popularity, na.rm = TRUE), canciones = n()) %>%
  arrange(desc(media_pop))

top_genres <- head(genre_stats, 10)

ggplot(top_genres, aes(x = reorder(track_genre, media_pop), y = media_pop)) +
  geom_col(fill = "purple") +
  coord_flip() +
  labs(title = "Top 10 géneros por popularidad media", x = "Género", y = "Popularidad media")


## 8️⃣ Top artistas por popularidad media

In [ ]:
top_artistas <- spotify %>%
  group_by(artists) %>%
  summarise(canciones = n(), popularidad_media = mean(popularity, na.rm = TRUE)) %>%
  arrange(desc(popularidad_media)) %>% head(10)

ggplot(top_artistas, aes(x = reorder(artists, popularidad_media), y = popularidad_media)) +
  geom_col(fill = "orange") +
  coord_flip() +
  labs(title = "Top 10 artistas por popularidad media", x = "Artista", y = "Popularidad media")


## 9️⃣ Conclusiones

- La mayoría de canciones duran entre 2 y 4 minutos.
- La popularidad se concentra en valores medios-altos.
- Los géneros con más energía tienden a ser más populares.
- Existen correlaciones moderadas entre energía, danceability y valence.
